In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
import time
import random

random.seed(1)

network = Sequential()
network.add(Dense(20, init = 'lecun_uniform', input_dim=21))
network.add(Activation('relu'))
network.add(Dense(3, init='lecun_uniform'))
network.add(Activation('sigmoid'))
network.compile(loss='mse', optimizer='sgd')

def trainNetwork(games, gamma, ep, start):
    for i in range(games):
        done = False
        if i%10000==0:
            print("game: ", i)
        total = start
        while True:
            innum1 = np.zeros((start,1))
            index = total - 1
            innum1[index] = 1
            innum1 = innum1.T
            outnum1 = network.predict(innum1, batch_size=1)
            if random.random()>ep:
                move_choice = np.argmax(outnum1[0])
            else:
                move_choice = np.random.randint(0,3)
            subtractor = move_choice + 1
            total -= subtractor
            #print(subtractor, total)
            reward = rewardFunction(total)
            if reward!=0:
                expected_out = outnum1
                expected_out[0][move_choice] = reward
                network.fit(innum1, expected_out, batch_size=1, nb_epoch=1, verbose=0)
                break
            if random.random()>0.4:
                rand_choice = np.random.randint(1,4)
            else:
                rand_choice = perfectPlayer(total)
            total -= rand_choice
            #print("R", rand_choice, total)
            reward = rewardFunction(total, Q=False)
            if reward!=0:
                expected_out = outnum1
                expected_out[0][move_choice] = reward
                network.fit(innum1, expected_out, batch_size=1, nb_epoch=1, verbose=0)
                break
            innum2 = np.zeros((start,1))
            index2 = total-1
            innum2[index2] = 1
            innum2 = innum2.T
            outnum2 = network.predict(innum2, batch_size=1)
            maxQ = max(outnum2[0])
            expected_out = outnum1
            expected_out[0][move_choice] = gamma*maxQ
            network.fit(innum1, expected_out, batch_size=1, nb_epoch=1, verbose=0)
        if ep > 0.1:
            ep -= 1/games
        if i>14000 and i%200==0:
            done = checkDoneTraining(start)
        if done:
            print('Training completed in ', i, 'games')
            break

def play(start, games):
    for i in range(games):
        total = start -1
        print("Total: ", total+1,"\n")
        while True:
            innum = np.zeros((start,1))
            innum[total] = 1
            innum = innum.T
            print(network.predict(innum, verbose=0))
            choice = np.argmax(network.predict(innum, verbose=0))
            total = total - choice -1
            print("Computer chooses: ",choice+1," Total remining: ",total+1,"\n")
            if checkGameOver(total):
                print("Computer Wins!!!\n")
                break
            while True:
                pchoice = int(input("Player Select 1, 2, or 3: "))
                if((pchoice != 1 and pchoice != 2 and pchoice !=3) or total-pchoice < -1):
                    print("Invalid Entry, Please select a legal move")
                else:
                    break
            total = total - pchoice
            print("Total remaining: ",total+1,"\n")
            if checkGameOver(total):
                print("Player Wins!!!\n")
                break
                
                
def checkDoneTraining(start):
    wins = 0
    for i in range(10):
        total = start
        while True:
            innum = np.zeros((start,1))
            innum[total-1] = 1
            innum = innum.T
            choice = np.argmax(network.predict(innum, verbose=0))
            #print('choice', choice+1)
            total = total - choice -1
            #print('total            ', total)
            reward = rewardFunction(total)
            if reward==100:
                wins += 1
                #print('win', wins, '\n')
                break
            pchoice = perfectPlayer(total)
            #print('perfectchoice', pchoice)
            total -= pchoice
            #print('total', total)
            reward = rewardFunction(total, Q=False)
            if reward==-100:
                #print('not done\n')
                wins = 0
                break
    if wins==10:
        return True
    else:
        return False
                
                
def perfectPlayer(total):
    if total%4==0:
        choice = np.random.randint(1,4)
    if total%4==1:
        choice = 1
    if total%4==2:
        choice = 2
    if total%4==3:
        choice = 3
    return choice   
                
def checkGameOver(total):
    if total <= 0:
        return True
    else:
        return False
            
def rewardFunction(number, Q=True):
    if Q:
        if number==0:
            return 100
        if number<0:
            return -1
    else:
        if number<=0:
            return -100
    return 0


games = 50000 #training takes about 120000 games for a game of 21
gamma = 0.99  #when it plays a perfect player it only takes about 30000 games
ep = 1
start = 21
start1 = time.time()
trainNetwork(games, gamma, ep, start)
end = time.time()
print(end-start1)

Using Theano backend.


game:  0
game:  10000
game:  20000
Training completed in  24600 games
77.75635170936584


In [2]:
play(21, 1)

Total:  21 

[[ 0.80782259  0.76126629  0.78950506]]
Computer chooses:  1  Total remining:  20 

Player Select 1, 2, or 3: 1
Total remaining:  19 

[[ 0.66170853  0.71660119  0.73837334]]
Computer chooses:  3  Total remining:  16 

Player Select 1, 2, or 3: 2
Total remaining:  14 

[[ 0.72717857  0.8614549   0.55956256]]
Computer chooses:  2  Total remining:  12 

Player Select 1, 2, or 3: 2
Total remaining:  10 

[[ 0.66058719  0.91397095  0.32064992]]
Computer chooses:  2  Total remining:  8 

Player Select 1, 2, or 3: 1
Total remaining:  7 

[[ 0.49389869  0.4342801   0.97915542]]
Computer chooses:  3  Total remining:  4 

Player Select 1, 2, or 3: 2
Total remaining:  2 

[[  1.45874947e-04   9.99998987e-01   1.44747579e-02]]
Computer chooses:  2  Total remining:  0 

Computer Wins!!!

